In [30]:
# Cogigo que aplica los modelos y genera la data de segmentación

import pandas as pd
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
import dill
import pickle
import os


def model_segmentation(file_rfm, file_segmento, t=180):

    clv = pd.read_csv(os.path.join('../data/processed', file_rfm))
    print(file_rfm, ' cargado correctamente')

    # Leemos los modelo entrenado para usarlo
    bgf_package = '../models/bgf_model.pkl'
    bgf_loaded = pickle.load(open(bgf_package, 'rb'))

    ggf_package = '../models/ggf_model.pkl'
    ggf_loaded = pickle.load(open(ggf_package, 'rb'))
    
    print('Modelos importado correctamente')

    # Number of purchases
    # t = 180 # 30 day period
    clv['expected_purc_6_months'] = \
    bgf_loaded.conditional_expected_number_of_purchases_up_to_time(t, clv['frequency'], clv['recency'], clv['T'])

    # Customer Lifetime Value
    clv['6_Months_CLV']=ggf_loaded.customer_lifetime_value(bgf_loaded,
                                                    clv["frequency"],
                                                    clv["recency"],
                                                    clv["T"],
                                                    clv["monetary_value"],
                                                    time=t/30,
                                                    freq='D',
                                                    discount_rate=0.01)

    clv.to_csv(f'../data/segments/{file_segmento}')
    
    print(f'Archivo de segmentación {file_segmento} generado y guardado')

    return clv

def main():
    
    df = model_segmentation(file_rfm='rfm_data.csv', file_segmento='segmentos.csv')
    print('Finalizó la segmentación de la base')

PRUEBAS

In [26]:
model_segmentation(file_rfm='rfm_data.csv', file_segmento='segmentos.csv')

rfm_data.csv  cargado correctamente
Modelos importado correctamente
Archivo de segmentación segmentos.csv generado y guardado


,CustomerID,frequency,recency,T,monetary_value,expected_purc_6_months,6_Months_CLV
0,12747.0,10.0,367.0,369.0,375.725000,4.632775,1712.381633
1,12748.0,112.0,373.0,373.0,257.314911,42.444401,10566.853327
2,12749.0,3.0,210.0,213.0,999.106667,2.979965,3054.158128
3,12820.0,3.0,323.0,326.0,256.573333,2.211293,584.100407
4,12823.0,4.0,222.0,296.0,252.450000,2.814953,719.842220
...,...,...,...,...,...,...,...
1733,18263.0,2.0,227.0,252.0,467.580000,2.166345,1074.547597
1734,18272.0,5.0,244.0,246.0,485.688000,3.710709,1805.476473
1735,18273.0,2.0,255.0,257.0,71.400000,2.136996,165.434190
1736,18283.0,13.0,334.0,337.0,152.436154,6.165368,921.670085


In [31]:
main()

rfm_data.csv  cargado correctamente
Modelos importado correctamente
Archivo de segmentación segmentos.csv generado y guardado
Finalizó el Scoring del Modelo
